<a href="https://colab.research.google.com/github/tontan2545/Intro_to_Data_Science/blob/main/Assignment_2_Voice_Essay_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import pickle
import os
from tqdm import tqdm

# Topic 2: Voice Processing

## Imports

In [105]:
import re
from scipy.io import wavfile

## Voice Preprocessing

In [149]:
data_dir = "/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data"
n_voice = 0

In [135]:
def time_points_processing(voice):
  mid_value = len(voice) // 2
  target = 44100
  return voice[mid_value - target//2: mid_value + target//2]

## Voice Processing

In [152]:
n_voice = 360
unique_owner = 0
file_idx = 0
owner2number = {}
data_array = np.zeros((n_voice, 44100))
cat_array = np.zeros((n_voice))
owner_array = np.zeros((n_voice))
for sub_dir, dirs, files in os.walk(data_dir):
  print(sub_dir)
  for file in files:
    name = re.search('[0-9]+_(.+?)_[0-9]+.wav', file).group(1).lower()
    cat_number = re.search(f'{data_dir}/(.*)_', sub_dir).group(1)
    _, data = wavfile.read(os.path.join(sub_dir, file))
    if(name not in owner2number):
      owner2number[name] = unique_owner
      unique_owner += 1
    data = time_points_processing(data)
    owner_array[file_idx] = owner2number[name]
    cat_array[file_idx] = cat_number
    data_array[file_idx] = np.array(data)
    file_idx += 1
data_array

/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/1_กา
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/1_กา/.ipynb_checkpoints
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/2_A
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/3_Monkey sound
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/4_C
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/10_CarEngine
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/5_มอ
/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Topic2: Standardized Voice Data/9_Meow
/content/gdrive/MyDrive/Intro_to_data_science/2 Da

array([[  -17.,   -12.,   -14., ...,   -17.,   -17.,   -17.],
       [   -3.,    -7.,    -6., ...,   -41.,   -40.,   -41.],
       [   45.,    22.,     5., ...,    64.,    74.,    89.],
       ...,
       [   -9.,    -6.,    -4., ...,   113.,   117.,   104.],
       [  126.,   129.,   118., ...,   -34.,   -35.,   -39.],
       [   -1.,    26.,    52., ..., -2246., -2249., -2196.]])

## Voice Serializing

In [154]:
voice_dest_dir = '/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Assignment 2'
voice_out_dir = os.path.join(voice_dest_dir, "topic_2_voice.pickle")
with open(voice_out_dir, 'wb') as file:
     pickle.dump(((data_array, cat_array, owner_array)), file)
     print("Dump succeed!")

Dump succeed!


# Topic 4: Essay Processing

## Imports

In [38]:
!pip install pythainlp
!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut

In [39]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [40]:
from pythainlp import word_tokenize

In [41]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [42]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Lil9Wb3zIkN9ahFIgj98ZlAZuQNmeaLyb1-1cakJ7ow/edit?usp=sharing')

In [43]:
sheet = wb.worksheet("Merged Table")

In [49]:
raw_essays = np.array(sheet.get_all_values())
raw_essays

array([['', 'Name count', '1', '2', '3', '4', '5', '6', '7', '8', '9',
        '10', '11', '12', '13', '14', '15', '16', '17', '18'],
       ['No.', 'Thai alphabets / Name', 'Phawat', 'Nakrit', 'Jomphat',
        'Waritnan', 'Pannaton', 'Panud', 'Thanapat', 'Sukit', 'Pinpinut',
        'Gritt', 'Jirayu', 'Supanut', 'Monthon', 'Nonthapat', 'Yanisa',
        'Phataraporn', 'Tasha', 'Naphatmanu'],
       ['1', 'ก',
        'ในช่วงที่หมูราคาแพง หลายคนคงหันมาซื้อไก่มากขึ้น ไก่แต่เดิมแล้วถ้าเทียบกับประเทศอื่น ไก่มีราคาที่ถูกกว่ามาก ส่วนต่างๆของไก่ที่ควรจะแพงอย่างเช่นอกไก่ ซึ่งเป็นแหล่งโปรตีนชั้นเยี่ยม มีราคาถูกเพราะอกไก่ไม่อร่อย เลยไม่นิยมในคนไทย แต่คนไทยก็ชอบไก่ทอดเป็นพิเศษ ตอนนี้ร้านไก่ทอดเกาหลีก็เริ่มจะล้นตลาดแล้ว',
        'ไก่เป็นสัตว์ปีก มีสองปีก สองขา บินไม่ได้ แต่อร่อย ไก่ต้มไม่อร่อย ไก่ทอดอร่อย ไก่ย่างก็อร่อย ไก่ทอดบอนชอนอร่อยมากแต่สู้ไก่ทอดหาดใหญ่ไม่ได้ที่ใส่หอมเจียวเยอะ ๆ อร่อยที่สุดในโลก อยากกินมาก',
        'ไก่เป็นสัตว์ปีกที่ออกลูกเป็นไข่ แม่ไก่ฟักไข่ออกมาเป็นลูกเจี๊ยบ ลูกเจี๊ย

## GG Sheet Preprocessing

In [50]:
essays = raw_essays[2:,2:]
essays

array([['ในช่วงที่หมูราคาแพง หลายคนคงหันมาซื้อไก่มากขึ้น ไก่แต่เดิมแล้วถ้าเทียบกับประเทศอื่น ไก่มีราคาที่ถูกกว่ามาก ส่วนต่างๆของไก่ที่ควรจะแพงอย่างเช่นอกไก่ ซึ่งเป็นแหล่งโปรตีนชั้นเยี่ยม มีราคาถูกเพราะอกไก่ไม่อร่อย เลยไม่นิยมในคนไทย แต่คนไทยก็ชอบไก่ทอดเป็นพิเศษ ตอนนี้ร้านไก่ทอดเกาหลีก็เริ่มจะล้นตลาดแล้ว',
        'ไก่เป็นสัตว์ปีก มีสองปีก สองขา บินไม่ได้ แต่อร่อย ไก่ต้มไม่อร่อย ไก่ทอดอร่อย ไก่ย่างก็อร่อย ไก่ทอดบอนชอนอร่อยมากแต่สู้ไก่ทอดหาดใหญ่ไม่ได้ที่ใส่หอมเจียวเยอะ ๆ อร่อยที่สุดในโลก อยากกินมาก',
        'ไก่เป็นสัตว์ปีกที่ออกลูกเป็นไข่ แม่ไก่ฟักไข่ออกมาเป็นลูกเจี๊ยบ ลูกเจี๊ยบตัวเล็กและมีสีเหลือง เนื้อไก่มีโปรตีนสูง นักเพาะกายนิยมกินอกไก่ปั่นหลังออกกำลังกาย เพราะในเนื้อไก่ โดยเฉพาะส่วนอก นั้นมีโปรตีนสูงและมีไขมันต่ำ ไก่นั้นมักจะขันในเวลาย่ำรุ่ง ก่อให้เกิดความรำคาญแก่คนที่ชอบนอนตอนเช้าเป็นอย่างยิ่ง',
        'การเลี้ยงไก่ไข่แบบปล่อยอิสระเป็นวิธีที่ได้รับความนิยมอย่างมาก เพราะจะทำให้ไก่อารมณ์ดีมีสุขภาพจิตใจที่ดี แข็งแรง ส่งผลดีกับผลผลิตที่จะได้ออกมา ทำให้ไข่แดงสีเข้ม ไข่ขาวสีข้นสวยงามช

## Essay preprocessing

In [72]:
to_remove = [
             '#N/A',
             'N/A',
             '\n',
             '\r',
             '\xa0',
             '\u2028',
             '#',
             '\u200b'
]
for i in range(essays.shape[0]):
  for j in range(essays.shape[1]):
    for char in to_remove:
      essays[i][j] = essays[i][j].replace(char, "")
essays

array([['ในช่วงที่หมูราคาแพง หลายคนคงหันมาซื้อไก่มากขึ้น ไก่แต่เดิมแล้วถ้าเทียบกับประเทศอื่น ไก่มีราคาที่ถูกกว่ามาก ส่วนต่างๆของไก่ที่ควรจะแพงอย่างเช่นอกไก่ ซึ่งเป็นแหล่งโปรตีนชั้นเยี่ยม มีราคาถูกเพราะอกไก่ไม่อร่อย เลยไม่นิยมในคนไทย แต่คนไทยก็ชอบไก่ทอดเป็นพิเศษ ตอนนี้ร้านไก่ทอดเกาหลีก็เริ่มจะล้นตลาดแล้ว',
        'ไก่เป็นสัตว์ปีก มีสองปีก สองขา บินไม่ได้ แต่อร่อย ไก่ต้มไม่อร่อย ไก่ทอดอร่อย ไก่ย่างก็อร่อย ไก่ทอดบอนชอนอร่อยมากแต่สู้ไก่ทอดหาดใหญ่ไม่ได้ที่ใส่หอมเจียวเยอะ ๆ อร่อยที่สุดในโลก อยากกินมาก',
        'ไก่เป็นสัตว์ปีกที่ออกลูกเป็นไข่ แม่ไก่ฟักไข่ออกมาเป็นลูกเจี๊ยบ ลูกเจี๊ยบตัวเล็กและมีสีเหลือง เนื้อไก่มีโปรตีนสูง นักเพาะกายนิยมกินอกไก่ปั่นหลังออกกำลังกาย เพราะในเนื้อไก่ โดยเฉพาะส่วนอก นั้นมีโปรตีนสูงและมีไขมันต่ำ ไก่นั้นมักจะขันในเวลาย่ำรุ่ง ก่อให้เกิดความรำคาญแก่คนที่ชอบนอนตอนเช้าเป็นอย่างยิ่ง',
        'การเลี้ยงไก่ไข่แบบปล่อยอิสระเป็นวิธีที่ได้รับความนิยมอย่างมาก เพราะจะทำให้ไก่อารมณ์ดีมีสุขภาพจิตใจที่ดี แข็งแรง ส่งผลดีกับผลผลิตที่จะได้ออกมา ทำให้ไข่แดงสีเข้ม ไข่ขาวสีข้นสวยงามช

## Essay Sequencing

In [83]:
max_token = max([len(word_tokenize(str(essay), engine="deepcut")) for essay in essays])
max_token

1312

In [91]:
essay_cnt = essays.reshape(-1).shape[0]
sequence_array = np.zeros((essay_cnt,max_token))
cat_array = np.zeros((essay_cnt,))
owner_array = np.zeros((essay_cnt,))
word2number = {}
number2word = {}
unique_words_cnt = 0
cnt = 0
for i in tqdm(range(len(essays))):
  for j in range(len(essays[i])):
    cat_array[cnt] = i
    owner_array[cnt] = j
    words = word_tokenize(essays[i][j], engine="deepcut")
    for word_idx in range(len(words)):
      word = words[word_idx]
      if(word not in word2number):
        unique_words_cnt += 1
        word2number[word] = unique_words_cnt
        number2word[unique_words_cnt] = word
      sequence_array[cnt, word_idx] = word2number[word]
    cnt += 1

100%|██████████| 26/26 [01:00<00:00,  2.34s/it]


{'ใน': 1,
 'ช่วง': 2,
 'ที่': 3,
 'หมู': 4,
 'ราคา': 5,
 'แพง': 6,
 ' ': 7,
 'หลาย': 8,
 'คน': 9,
 'คง': 10,
 'หัน': 11,
 'มา': 12,
 'ซื้อ': 13,
 'ไก่': 14,
 'มาก': 15,
 'ขึ้น': 16,
 'แต่': 17,
 'เดิม': 18,
 'แล้ว': 19,
 'ถ้า': 20,
 'เทียบ': 21,
 'กับ': 22,
 'ประเทศ': 23,
 'อื่น': 24,
 'มี': 25,
 'ถูก': 26,
 'กว่า': 27,
 'ส่วน': 28,
 'ต่างๆ': 29,
 'ของ': 30,
 'ควร': 31,
 'จะ': 32,
 'อย่าง': 33,
 'เช่น': 34,
 'อก': 35,
 'ซึ่ง': 36,
 'เป็น': 37,
 'แหล่ง': 38,
 'โปรตีน': 39,
 'ชั้น': 40,
 'เยี่ยม': 41,
 'เพราะ': 42,
 'ไม่': 43,
 'อร่อย': 44,
 'เลย': 45,
 'นิยม': 46,
 'ไทย': 47,
 'ก็': 48,
 'ชอบ': 49,
 'ทอด': 50,
 'พิเศษ': 51,
 'ตอน': 52,
 'นี้': 53,
 'ร้าน': 54,
 'เกาหลี': 55,
 'เริ่ม': 56,
 'ล้น': 57,
 'ตลาด': 58,
 'สัตว์': 59,
 'ปีก': 60,
 'สอง': 61,
 'ขา': 62,
 'บิน': 63,
 'ได้': 64,
 'ต้ม': 65,
 'ทอดอร่อย': 66,
 'ย่าง': 67,
 'บอน': 68,
 'ชอน': 69,
 'สู้': 70,
 'หาดใหญ่': 71,
 'ใส่': 72,
 'หอม': 73,
 'เจียว': 74,
 'เยอะ': 75,
 'ๆ': 76,
 'ที่สุด': 77,
 'โลก': 78,
 'อยาก': 79,
 'กิน': 80

In [96]:
sequence_array


array([[   1.,    2.,    3., ...,    0.,    0.,    0.],
       [  14.,   37.,   59., ...,    0.,    0.,    0.],
       [  14.,   37.,   59., ...,    0.,    0.,    0.],
       ...,
       [3125.,  277., 2040., ...,    0.,    0.,    0.],
       [3102.,  186.,  101., ...,    0.,    0.,    0.],
       [3132.,   37.,   59., ...,    0.,    0.,    0.]])

## Essay Serializing

In [98]:
essay_dest_dir = '/content/gdrive/MyDrive/Intro_to_data_science/2 Data Preparation/Assignment 2'
essay_out_dir = os.path.join(essay_dest_dir, "topic_4_essay.pickle")
with open(essay_out_dir, 'wb') as file:
     pickle.dump(((sequence_array, cat_array, owner_array, number2word)), file)
     print("Dump succeed!")

Dump succeed!
